Lab 03: VGG

In [1]:
#import các thư viện
import keras
from keras import backend as k
import tensorflow as tf
import numpy as np
from keras import applications
from keras.datasets import cifar10
#chọn hệ số lamda
regularizer=0.0005

### 1. Hãy dùng thư viện keras.layers để cài đặt VGG19 huấn luyện trên bộ dữ liệu cifar-10

In [2]:
## tải CIFAR-100 dataset từ keras
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

##resacle ảnh thành ảnh thực trong đoạn [0,1]
X_train, X_test = X_train/255.0, X_test/255.0

import cv2
## Resize toàn bộ ảnh train tập train và test
X_train = np.array([cv2.resize(img, dsize=(32,32)) for img in X_train])
X_test = np.array([cv2.resize(img, dsize=(32,32)) for img in X_test])
print("Kích thước tập sau khi resize: ", X_train.shape, X_test.shape)

#Tách một phần tập train thành tập valid
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1)

170498071/170498071 [==============================] - 2s 0us/step
Kích thước tập sau khi resize:  (50000, 32, 32, 3) (10000, 32, 32, 3)


Cấu trúc mô hình

In [3]:
#Dùng thư viện
vgg19 = keras.applications.vgg19.VGG19(
        include_top=True,
        weights=None,
        input_tensor=None,
        input_shape= (32,32,3),
        pooling=None,
        classes=10,
        classifier_activation="softmax",
)
vgg19.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0     

In [4]:
#Khởi tạo mô hình
model=keras.Sequential()

#input_shape
model.add(keras.layers.Input(shape=(32,32,3)))

#block1
#Gồm 2 lớp Convolution và 1 lớp maxpooling pool_size là 2x2
#filter 64, kernel_size là 3x3 và stride 1x1, có padding, activation là relu
model.add(keras.layers.Convolution2D(filters=64,
                                     kernel_size=[3,3],
                                     strides=[1,1],
                                     padding="same",
                                     activation=keras.activations.relu))
model.add(keras.layers.Convolution2D(filters=64,
                                     kernel_size=[3,3],
                                     strides=[1,1],
                                     padding='same',
                                     activation=keras.activations.relu))
model.add(keras.layers.MaxPool2D(pool_size=[2,2],
                                 strides=[2,2]))

#block2
#Gồm 2 lớp Convolution và 1 lớp maxpooling với pool_size là 2x2
#filter 128, kernel_size là 3x3 và stride 1x1, có padding, activation là relu
model.add(keras.layers.Convolution2D(filters=128,
                                     kernel_size=[3,3],
                                     strides=[1,1],
                                     padding="same",
                                     activation=keras.activations.relu))
model.add(keras.layers.Convolution2D(filters=128,
                                     kernel_size=[3,3],
                                     strides=[1,1],
                                     padding="same",
                                     activation=keras.activations.relu))
model.add(keras.layers.MaxPool2D(pool_size=[2,2],
                    strides=[2,2]))

#block3
#Gồm 3 lớp Convolution và 1 lớp maxpooling với pool_size là 2x2
#filter 256, kernel_size là 3x3 và stride 1x1, có padding, activation là relu
model.add(keras.layers.Convolution2D(filters=256,
                                     kernel_size=[3,3],
                                     strides=[1,1],
                                     padding="same",
                                     activation=keras.activations.relu))
model.add(keras.layers.Convolution2D(filters=256,
                                     kernel_size=[3,3],
                                     strides=[1,1],
                                     padding="same",
                                     activation=keras.activations.relu))
model.add(keras.layers.Convolution2D(filters=256,
                                     kernel_size=[3,3],
                                     strides=[1,1],
                                     padding="same",
                                     activation=keras.activations.relu))
model.add(keras.layers.Convolution2D(filters=256,
                                     kernel_size=[3,3],
                                     strides=[1,1],
                                     padding="same",
                                     activation=keras.activations.relu))
model.add(keras.layers.MaxPool2D(pool_size=[2,2],
                          strides=[2,2]))
#block4
#Gồm 3 lớp Convolution và 1 lớp maxpooling với pool_size là 2x2
#filter 512, kernel_size là 3x3 và stride 1x1, có padding, activation là relu
model.add(keras.layers.Convolution2D(filters=512,
                                     kernel_size=[3,3],
                                     strides=[1,1],
                                     padding="same",
                                     activation=keras.activations.relu))
model.add(keras.layers.Convolution2D(filters=512,
                                     kernel_size=[3,3],
                                     strides=[1,1],
                                     padding="same",
                                     activation=keras.activations.relu))
model.add(keras.layers.Convolution2D(filters=512,
                                     kernel_size=[3,3],
                                     strides=[1,1],
                                     padding="same",
                                     activation=keras.activations.relu))
model.add(keras.layers.Convolution2D(filters=512,
                                     kernel_size=[3,3],
                                     strides=[1,1],
                                     padding="same",
                                     activation=keras.activations.relu))
model.add(keras.layers.MaxPool2D(pool_size=[2,2],
                                 strides=[2,2]))
#block5
#Gồm 4 lớp Convolution và 1 lớp maxpooling với pool_size là 2x2
#filter 512, kernel_size là 3x3 và stride 1x1, có padding, activation là relu
model.add(keras.layers.Convolution2D(filters=512,
                                     kernel_size=[3,3],
                                     strides=[1,1],
                                     padding="same",
                                     activation=keras.activations.relu))
model.add(keras.layers.Convolution2D(filters=512,
                                     kernel_size=[3,3],
                                     strides=[1,1],
                                     padding="same",
                                     activation=keras.activations.relu))
model.add(keras.layers.Convolution2D(filters=512,
                                     kernel_size=[3,3],
                                     strides=[1,1],
                                     padding="same",
                                     activation=keras.activations.relu))
model.add(keras.layers.Convolution2D(filters=512,
                                     kernel_size=[3,3],
                                     strides=[1,1],
                                     padding="same",
                                     activation=keras.activations.relu))
model.add(keras.layers.MaxPool2D(pool_size=[2,2],
                                 strides=[2,2]))

#block 6
#Gồm lớp Flatten để duỗi ra
model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(units=4096,
                             activation=keras.activations.relu,
                             kernel_regularizer=keras.regularizers.l2(regularizer)))
model.add(keras.layers.Dropout(rate=0.5))

model.add(keras.layers.Dense(units=4096,
                             activation=keras.activations.relu,
                             kernel_regularizer=keras.regularizers.l2(regularizer)))
model.add(keras.layers.Dropout(rate=0.5))

model.add(keras.layers.Dense(units=10,
                             activation=keras.activations.softmax))

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),    ##tự khai báo Optimizer với learning rate 10^-4
             loss=tf.keras.losses.sparse_categorical_crossentropy,
             metrics=["accuracy"])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 64)        1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 64)        36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 64)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 128)       73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 16, 16, 128)       147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 8, 128)        0         
 2D)                                                    

### 2. Chỉnh các tham số (lrn_rate, l2 weight decay, epochs, batch_size) và nhận xét performance sẽ thay đổi như thế nào

In [5]:
### Tạo một callback ModelCheckpoint: callback này sẽ lưu model mỗi khi tìm được một model tốt hơn
#### filepath: đường dẫn file muốn lưu
#### monitor: đại lượng quyết định xem model có "tốt hơn" hay không
#### mode='auto'/'max'/'min': đại lượng monitor lớn hay nhỏ là tốt hơn
#### verbose: có thông báo mỗi lần lưu ko
#### save_best_only: chỉ lưu model tốt nhất
mc = keras.callbacks.ModelCheckpoint(filepath="vgg19_cifar10.h5", 
                                     monitor='val_loss',
                                     mode='min', 
                                     verbose=0, 
                                     save_best_only=True)

## Train  ## Khuyến cáo chạy COLAB (hoặc tương tự)
history = model.fit(X_train, y_train,
                    batch_size=124,
                    epochs= 50,
                    validation_data=(X_valid, y_valid),
                    callbacks=[mc])                     ##sử dụng callback ModelCheckpoint trong quá trình train


## Đánh giá model trên tập test
valid_loss, valid_acc = model.evaluate(X_valid, y_valid)
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Valid: loss {} acc {} -- Test: loss {} valid {}".format(valid_loss, valid_acc, test_loss, test_acc))

## Load lại model tốt nhất đã lưu
print("best model: ")
model.load_weights("vgg19_cifar10.h5")
valid_loss, valid_acc = model.evaluate(X_valid, y_valid)
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Valid: loss {} acc {} -- Test: loss {} acc {}".format(valid_loss, valid_acc, test_loss, test_acc))

Epoch 1/50
363/363 [==============================] - 43s 89ms/step - loss: 2.9255 - accuracy: 0.1805 - val_loss: 2.1469 - val_accuracy: 0.2382
Epoch 2/50
363/363 [==============================] - 31s 85ms/step - loss: 1.9870 - accuracy: 0.2800 - val_loss: 1.7317 - val_accuracy: 0.3820
Epoch 3/50
363/363 [==============================] - 31s 85ms/step - loss: 1.6211 - accuracy: 0.4213 - val_loss: 1.5807 - val_accuracy: 0.4546
Epoch 4/50
363/363 [==============================] - 31s 85ms/step - loss: 1.3692 - accuracy: 0.5234 - val_loss: 1.2935 - val_accuracy: 0.5556
Epoch 5/50
363/363 [==============================] - 31s 84ms/step - loss: 1.1718 - accuracy: 0.5946 - val_loss: 1.1246 - val_accuracy: 0.6180
Epoch 6/50
363/363 [==============================] - 31s 85ms/step - loss: 1.0104 - accuracy: 0.6602 - val_loss: 1.0299 - val_accuracy: 0.6646
Epoch 7/50
363/363 [==============================] - 31s 84ms/step - loss: 0.8876 - accuracy: 0.7078 - val_loss: 0.9387 - val_accuracy:

In [7]:
import pandas as pd

d = {'epochs': [5,10,10,10,25,50, 200], 'batch_size': [10,124,124,124,64,124,128],
     'learning_rate':[0.001,0.0001,0.1,0.001,0.001,0.0001,0.001], 'decay weight': [0,0.0005,0.0005,0.0005,0,0.0005,0],
     'accuracy (%)':[10,70,10,10,10,86,87]}
df = pd.DataFrame(data=d)
df

,epochs,batch_size,learning_rate,decay weight,accuracy (%)
0,5,10,0.0010,0.0000,10
1,10,124,0.0010,0.0005,70
2,10,124,0.1000,0.0005,10
3,10,124,0.0010,0.0005,10
4,25,64,0.0010,0.0000,10
5,50,124,0.0001,0.0005,86
6,200,128,0.0010,0.0000,87


Nhìn vào bảng trên ta thấy với:
-  epochs = 50, batch_size = 124, learning rate = 0.0001 và L2 weight decay = 0.0005
- epochs = 200 và batch_size = 128, learning rate = 0.0001 và L2 weight decay = 0

Cả 2 đạt accuracy trên 85%, cho ta hiệu suất mô hình rất tốt. 

- epochs = 10, batch_size = 124, learning rate = 0.001 và L2 weight decay = 0.0005 cũng đạt accuracy là 70%, hiệu suất mô hình cũng khá tốt. 